In [275]:
import pandas as pd

## 处理jester_ratings.dat

In [277]:

jester_file_2='/Users/dong/Desktop/BoostingFM-IJCAI18/dataset/jester_dataset_2/jester_ratings.dat'
org_data=pd.read_table(jester_file,names=['user','vald1','item','vlad2','rating'])
del org_data['vald1']
del org_data['vlad2']
users=org_data['user'].drop_duplicates()
items=org_data['item'].drop_duplicates()
print 'users={0}'.format(len(users))
print 'items={0}'.format(len(items))
org_data.describe()

users=59132
items=140


,user,item,rating
count,1.761439e+06,1.761439e+06,1.761439e+06
mean,3.272322e+04,7.071133e+01,1.618602e+00
std,1.828011e+04,4.600790e+01,5.302608e+00
min,1.000000e+00,5.000000e+00,-1.000000e+01
25%,1.720200e+04,2.100000e+01,-2.031000e+00
50%,3.480800e+04,6.900000e+01,2.219000e+00
75%,4.730600e+04,1.120000e+02,5.719000e+00
max,6.397800e+04,1.500000e+02,1.000000e+01


In [278]:
lt1=len(org_data[org_data['rating']>1]) #1047673
lt3=len(org_data[org_data['rating']>3]) 
lt5=len(org_data[org_data['rating']>5]) 
print lt1*1./total,lt3*1./total,lt5*1./total

0.59478244776 0.438956444135 0.294790793209


In [279]:
group_by_user=org_data.groupby('user').count()

In [280]:
items_gt_10=group_by_user[group_by_user['item']>10]
items_gt_30=group_by_user[group_by_user['item']>30]
items_gt_50=group_by_user[group_by_user['item']>50]
items_gt_100=group_by_user[group_by_user['item']>100]

In [281]:
print len(items_gt_10), len(items_gt_30),len(items_gt_50),len(items_gt_100)

39047 17610 10291 4206


# 
1. 从item大于50的用户中随机选取1000个
2. 挑选出这1000个用户的数据
3. 按照user排序，将这1000个用户的

In [189]:
all_users_id=items_gt_50.index.values
print len(all_users_id)

10291


In [283]:
idx=range(len(all_users_id))
np.random.shuffle(idx)
target_users_1000=items_gt_50.index.values[idx][:1000]
# print idx

In [284]:
df_list=[]
for user in target_users_1000:
    temp=org_data[org_data['user']==user]
    df_list.append(temp)
final_result_df  = pd.concat(df_list,ignore_index=True)

In [285]:
final_result_df=final_result_df.sort_values(by=['user'])
new_user_id=range(1000)
old_user_id=np.sort(target_users_1000)
replace_dic={}
for new_id,old_id in zip(new_user_id,old_user_id):
    replace_dic[old_id] = new_id
new_user_id=final_result_df['user'].replace(replace_dic) 
final_result_df['user']=new_user_id

In [249]:
final_result_df.to_csv('final_result_df.csv',index=False,sep='|',header=False)

In [286]:

positive=final_result_df[final_result_df['rating']>=5] # 28466
positive.to_csv('rating_gth_5.csv',index=False,sep='|',header=False)
# positive=positive.sample(frac=1)

In [290]:

train_data=[]
test_data=[] 
user_count=0
replace_dict={}
for u in range(1000):
    temp=positive[positive['user']==u]
    count=len(temp)
    if count < 10:
        continue
    user_count+=1
    replace_dict[u]=user_count
    tr_num=int(np.ceil(0.8*count))
    train_data.append(temp[:tr_num])
    test_data.append(temp[tr_num:])


In [291]:
train_50_5_data=pd.concat(train_data)
test_50_5_data=pd.concat(test_data)

In [303]:
temp_train_rp=train_50_5_data['user'].replace(replace_dict)
temp_test_rp=test_50_5_data['user'].replace(replace_dict)
train_50_5_data['user'] = temp_train_rp
test_50_5_data['user']  = temp_test_rp



In [306]:
train_50_5_data.to_csv('train_50_5_data.dat',sep='|',index=None,header=None)
test_50_5_data.to_csv('test_50_5_data.dat',sep='|',index=None,header=None)


In [308]:
train_len=len(train_50_5_data)
test_len=len(test_50_5_data)
temp_rating_tr=pd.Series(np.array([1.]*train_len))
temp_rating_tes=pd.Series(np.array([1.]*test_len))

train_50_5_data['user']=temp_rating_tr
temp_rating_tes['user']=temp_rating_tes



In [314]:
org_data.describe()

,user,item,rating
count,1.761439e+06,1.761439e+06,1.761439e+06
mean,3.272322e+04,7.071133e+01,1.618602e+00
std,1.828011e+04,4.600790e+01,5.302608e+00
min,1.000000e+00,5.000000e+00,-1.000000e+01
25%,1.720200e+04,2.100000e+01,-2.031000e+00
50%,3.480800e+04,6.900000e+01,2.219000e+00
75%,4.730600e+04,1.120000e+02,5.719000e+00
max,6.397800e+04,1.500000e+02,1.000000e+01


In [315]:
test_50_5_data.describe()

,user,item,rating
count,5159.000000,5159.000000,5159.000000
mean,364.446211,86.753441,7.595926
std,208.660589,39.147959,1.550232
min,1.000000,5.000000,5.000000
25%,178.000000,53.000000,6.219000
50%,371.000000,91.000000,7.562000
75%,555.500000,120.000000,9.031000
max,702.000000,150.000000,10.000000


In [ ]:
train_50_5_data.to_csv('train_50_5_data_0_1_coo.dat',sep='\t',index=None,header=None)
test_50_5_data.to_csv('test_50_5_data_0_1_coo.dat',sep='\t',index=None,header=None)

In [259]:
train_50_5_data

TypeError: 'int' object has no attribute '__getitem__'

In [172]:
final_result_df.sort_values(by=['user'])

,user,item,rating
1131736,0,31,8.781
1131749,0,68,-0.125
1131750,0,81,0.125
1131751,0,61,0.031
1131752,0,26,0.031
1131753,0,92,3.625
1131754,0,118,0.000
1131755,0,134,3.344
1131717,0,108,5.688
1131716,0,107,2.031


In [143]:

target_uses_id = pd.concat()



np.sum(target_users_1000)-np.sum(items_gt_50.index.values[:100])


30930589

array([40978,  3250, 47484, ...,  7521, 23342, 35439])

In [160]:
len(items_gt_50)

10291

In [125]:
final_result_df.sort_values(by=['user']) 
final_result_df.to_csv('final_result_df_org_u1000_i140.dat',index=False)

,user,item,rating
37180,16,68,-9.438
37134,16,16,-8.938
37133,16,15,-9.938
37132,16,13,-9.906
37131,16,8,1.562
37130,16,7,-9.938
37129,16,5,0.469
37179,16,56,-9.906
37165,16,31,9.938
37166,16,89,1.312


In [115]:
for user in target_uses_id:
    temp=org_data[org_data['user']==user]

array([    1,     5,     8, ..., 63949, 63950, 63978])

ValueError: Lengths must match to compare

In [100]:
org_data[:60]

,user,item,rating
0,1,5,0.219
1,1,7,-9.281
2,1,8,-9.281
3,1,13,-6.781
4,1,15,0.875
5,1,16,-9.656
6,1,17,-9.031
7,1,18,-7.469
8,1,19,-8.719
9,1,20,-9.156


In [ ]:
positive=org_data[org_data['rating']>=5]
neg=org_data[org_data['rating']<5]
output='/Users/dong/PycharmProjects/boosting2block_fm/data/data_set/jester-2'
jester_2_train_5.to_csv('jester_2_train_5.dat',sep='|',index=None,header=None)
jester_2_test_5.to_csv('jester_2_test_5.dat',sep='|',index=None,header=None)